In [1]:
from HMM_Functions import *
from matplotlib import cm, pyplot as plt
import seaborn
from matplotlib.dates import YearLocator, MonthLocator, DayLocator
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
def filtering(tpm, epm, pi, emissions):
    predict = []
    
    T = len(emissions)
    NUM_STATES = len(tpm)
    
    forwrd = forward(tpm, epm, pi, emissions[:-1])
    bkwrd = backward(tpm, epm, pi, emissions)
    
    
    normalization_const = #something
    
    for state_i in range(NUM_STATES):
        a = bkwrd[T][state_i]

        b = 0
        for state_j in range(NUM_STATES):
            b += tpm[state_j][state_j+1] * forwrd[T-1][state_j]
        
        predict.append(normalization_const * a * b)
    
    return predict

SyntaxError: invalid syntax (<ipython-input-3-15a688e2bd0d>, line 11)